# Generating Qubit Hamiltonians

In [10]:
from utility import *
import numpy as np 

# We define pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [2]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(h2))

H:
-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099508 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


This Qubit Hamiltonian encodes all of the $2^N$ eigenstates and eigenenergies, where $N=4$ is the number of qubits. 
In principle, one can represent the Hamiltonian as a matrix with $2^N$ dimensions and find the eigenvalues classically through diagonalization, but the cost of this approach clearly grows exponentially with $N$. 

Still, we can check that this approach indeed recovers the ground state energy against method in S1. 

In [3]:
# Build matrix representation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h2)
h2_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in h2.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h2_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h2_matrix)
print("The ground state energy from S1: ")
obtain_PES('h2', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -1.1011503301329566 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-1.10115033 -0.74587179 -0.74587179 -0.74587179 -0.60860671 -0.60860671
 -0.58166697 -0.58166697 -0.35229063 -0.06021529 -0.06021529 -0.05994379
 -0.05994379  0.03904763  0.50196598  0.52917721]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [4]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841725 [Z0]


We can verify that this new Hamiltonian still encodes the ground state energy. 

In [5]:
# Building the matrix representation of the effective Hamiltonian
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("The eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]


## LiH

### Using Jordan-Wigner transformation

In [6]:
qubit_transf = 'jw' # Jordan-Wigner transformations
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(lih))

H:
-3.934441956757901 [] +
-0.007923321157850174 [X0 X1 Y2 Y3] +
-0.0034145323580159117 [X0 X1 Y2 Z3 Z4 Y5] +
-0.0027468613203170036 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0034145323580159117 [X0 X1 X3 X4] +
-0.0027468613203170036 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.0048647783817614134 [X0 X1 Y4 Y5] +
-0.0022963165872996014 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0022963165872996014 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.0024727061683852834 [X0 X1 Y6 Y7] +
-0.0024727061683852847 [X0 X1 Y8 Y9] +
-0.0017744350099502865 [X0 X1 Y10 Y11] +
0.007923321157850174 [X0 Y1 Y2 X3] +
0.0034145323580159117 [X0 Y1 Y2 Z3 Z4 X5] +
0.0027468613203170036 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0034145323580159117 [X0 Y1 Y3 X4] +
-0.0027468613203170036 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.0048647783817614134 [X0 Y1 Y4 X5] +
0.0022963165872996014 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0022963165872996014 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.0024727061683852834 [X0 Y1 Y6 X7] +
0.0024727061683852847 [X0 Y1 Y8

In [7]:
# Build matrix representation of the Hamiltonian H
n_qubits = openfermion.count_qubits(lih)
lih_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in lih.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    lih_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(lih_matrix)
print("The ground state energy from S1: ")
obtain_PES('lih', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -7.784460280267016 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-7.78446028 -7.71129396 -7.71129396 ...  1.58753163  1.86602235
  2.08130331]


In [8]:
for i in range(len(eigvals)):
    print(eigvals[i])

-7.78446028003124
-7.711293961914158
-7.711293961914153
-7.658932360994443
-7.658932360994436
-7.65893236099443
-7.64449884290319
-7.635922969017906
-7.635922969017906
-7.635922969017903
-7.635922969017901
-7.618560726363894
-7.618560726363892
-7.618560726363891
-7.618560726363891
-7.618560726363891
-7.618560726363888
-7.585911678611021
-7.585911678611019
-7.551192161020098
-7.551192161020095
-7.551192161020093
-7.551192161020093
-7.551192161020093
-7.551192161020091
-7.551192161020091
-7.551192161020089
-7.533728529770468
-7.53372852977046
-7.502320520382563
-7.502320520382561
-7.502320520382561
-7.502320520382558
-7.494174261866586
-7.494174261866584
-7.4850637063362875
-7.485063706336282
-7.485063706336282
-7.485063706336277
-7.441500873919519
-7.4415008739195185
-7.441500873919515
-7.441500873919512
-7.431604098167831
-7.431604098167831
-7.431109575559062
-7.431109575559058
-7.431109575559054
-7.431109575559049
-7.368954803836322
-7.368954803836322
-7.326181290549242
-7.31587310734

-4.527988692252461
-4.527988692252459
-4.527988692252458
-4.527988692252457
-4.527988692252455
-4.52520445689717
-4.5252044568971685
-4.525204456897166
-4.525204456897165
-4.524466465946614
-4.5244664659466105
-4.524466465946603
-4.5244664659466025
-4.524466465946599
-4.524466465946599
-4.524466465946596
-4.524466465946595
-4.523907166231071
-4.523907166231071
-4.52390716623107
-4.523907166231069
-4.523907166231068
-4.5239071662310675
-4.519637433280114
-4.519637433280108
-4.519637433280107
-4.51136946412334
-4.511369464123336
-4.507945333513499
-4.507945333513494
-4.497137675252769
-4.497137675252768
-4.497137675252767
-4.497137675252765
-4.497137675252764
-4.497137675252764
-4.489881531124362
-4.48988153112436
-4.482940567457736
-4.482186441419784
-4.482186441419773
-4.482186441419773
-4.482186441419771
-4.482186441419771
-4.4821864414197705
-4.48218644141977
-4.48218644141977
-4.482186441419769
-4.482186441419763
-4.481395746031829
-4.481395746031825
-4.481395746031823
-4.4813957460

-3.92282000421025
-3.9171346552386503
-3.917134655238649
-3.917134655238648
-3.8870755779352253
-3.885681709469378
-3.8856817094693765
-3.8856817094693756
-3.8856817094693747
-3.8856817094693743
-3.8856817094693734
-3.8856817094693716
-3.885681709469371
-3.880826858771498
-3.8808268587714956
-3.8808268587714956
-3.880826858771495
-3.8794653910690275
-3.879465391069027
-3.879465391069027
-3.879465391069026
-3.879465391069026
-3.879465391069024
-3.8792917510633065
-3.8792917510633056
-3.8792917510633047
-3.8792917510633043
-3.8617557671352287
-3.8617557671352274
-3.8612742041110253
-3.8612742041110244
-3.8612742041110244
-3.8612742041110244
-3.8612742041110235
-3.861274204111022
-3.8612742041110204
-3.86127420411102
-3.8579084508056734
-3.8579084508056734
-3.8579084508056725
-3.8579084508056694
-3.8507305540118097
-3.850730554011808
-3.8367946765259573
-3.836794676525956
-3.836794676525955
-3.8295892763352426
-3.8282803133185963
-3.828280313318596
-3.828280313318596
-3.828280313318594
-3

-1.3175102981972213
-1.317510298197221
-1.3171077498889812
-1.31710774988898
-1.31710774988898
-1.3171077498889787
-1.317107749888975
-1.3171077498889736
-1.3128300784162377
-1.312830078416237
-1.3089299906604581
-1.308929990660457
-1.3089299906604568
-1.3089299906604512
-1.290085282561156
-1.2900852825611553
-1.2900852825611546
-1.2900852825611542
-1.2831036329707064
-1.283103632970706
-1.2831036329707057
-1.2831036329707055
-1.283103632970705
-1.2831036329707028
-1.2818656750903417
-1.281865675090341
-1.2747975127933304
-1.274797512793327
-1.2747975127933244
-1.2744932005780658
-1.274493200578065
-1.2744932005780643
-1.2744932005780636
-1.2744932005780627
-1.2744932005780616
-1.2708414725277133
-1.2708414725277122
-1.27084147252771
-1.2665300033027946
-1.2635416082257518
-1.2635416082257496
-1.2635416082257491
-1.2635416082257485
-1.2588024193310574
-1.258802419331056
-1.258802419331053
-1.2583376054111124
-1.258337605411112
-1.2583376054111102
-1.2583376054111082
-1.2583376054111062

### Using Bravyi-Kitaev transformation

In [24]:
qubit_transf = 'bk' #transformation
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(lih))

H:
-3.934441956757901 [] +
0.04069606505875819 [X0 X1 X2] +
-0.0031369437172315585 [X0 X1 X2 X3 Y7 Y11] +
0.0016354047160371907 [X0 X1 X2 Y3 Y5] +
0.0037144848216505294 [X0 X1 X2 Z3] +
0.0017809712977740805 [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
0.0047911716564078485 [X0 X1 Z2 Y3 Y4 X5] +
-0.00129194416648752 [X0 X1 X3 X4 Y7 Y11] +
0.001095485931572669 [X0 X1 X3 Y4 Y5 Z6 Z7] +
0.00129194416648752 [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
0.0025623902894191175 [X0 X1 X3 Z4 Y5 Y6 Z7] +
8.04917124090585e-05 [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
0.0013724358788965785 [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
0.0009084689845792891 [X0 X1 X3 X6 Y7 Y11] +
-8.981652044146124e-05 [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
0.0009084689845792898 [X0 X1 X3 Y7 X8 Y11] +
-0.0009084689845792898 [X0 X1 X3 Y7 Y8 Z10 X11] +
-8.981652044146084e-05 [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
0.0008186524641378289 [X0 X1 X3 Y7 Z8 Y10 X11] +
0.017297111110348394 [X0 X1 X3 Y7 Z9 Y10 X11] +
-0.002519150880579575 [X0 X1 X3 Y7 X10 Y11] +
-0.0008144678357783922 [X0 X1 Y3 X

In [25]:
# Build matrix representation of the Hamiltonian H
n_qubits = openfermion.count_qubits(lih)
lih_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in lih.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    lih_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(lih_matrix)
print("The ground state energy from S1: ")
obtain_PES('lih', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -7.784460280267016 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-7.78446028 -7.71129396 -7.71129396 ...  1.58753163  1.86602235
  2.08130331]


In [26]:
for i in range(len(eigvals)):
    print(eigvals[i])

-7.784460280031238
-7.711293961914156
-7.711293961914153
-7.658932360994442
-7.6589323609944415
-7.658932360994436
-7.644498842903193
-7.635922969017904
-7.635922969017902
-7.6359229690179005
-7.635922969017898
-7.618560726363895
-7.618560726363893
-7.618560726363893
-7.618560726363893
-7.6185607263638895
-7.6185607263638895
-7.585911678611019
-7.585911678611018
-7.551192161020098
-7.551192161020092
-7.551192161020089
-7.551192161020089
-7.551192161020088
-7.551192161020088
-7.551192161020087
-7.551192161020085
-7.533728529770472
-7.53372852977047
-7.502320520382563
-7.50232052038256
-7.502320520382559
-7.502320520382557
-7.494174261866584
-7.494174261866577
-7.48506370633629
-7.485063706336285
-7.4850637063362795
-7.4850637063362795
-7.4415008739195185
-7.441500873919516
-7.441500873919514
-7.441500873919513
-7.4316040981678375
-7.431604098167831
-7.431109575559057
-7.431109575559055
-7.431109575559055
-7.43110957555905
-7.3689548038363215
-7.3689548038363215
-7.3261812905492425
-7.31

-4.675485638231477
-4.672174863089232
-4.6721748630892295
-4.672174863089223
-4.670745247060723
-4.670745247060723
-4.668800088265219
-4.668800088265216
-4.665322089006069
-4.665322089006068
-4.665322089006066
-4.665322089006061
-4.663689665860631
-4.663689665860626
-4.662206469385043
-4.662206469385043
-4.662206469385041
-4.662206469385041
-4.654901503701157
-4.654901503701156
-4.654901503701156
-4.654901503701155
-4.654901503701154
-4.654901503701145
-4.649142859003371
-4.649142859003367
-4.646535306416445
-4.646535306416445
-4.646535306416444
-4.646535306416442
-4.64653530641644
-4.646535306416439
-4.645340767780709
-4.6453407677807075
-4.6453407677807075
-4.645340767780706
-4.645340767780706
-4.645340767780701
-4.6445475537609155
-4.644547553760914
-4.644547553760913
-4.642562542735302
-4.6425625427353
-4.642393473360335
-4.642393473360335
-4.6423934733603325
-4.639232544629014
-4.639232544629012
-4.639232544629011
-4.635779102933754
-4.635779102933753
-4.633453484074981
-4.6330686

-1.419748273600902
-1.4197482736009015
-1.4197482736009013
-1.4197482736009013
-1.4197482736009008
-1.4051791655740824
-1.4051791655740795
-1.4051791655740784
-1.4051791655740775
-1.3993084241551512
-1.3984963914641053
-1.3984963914641038
-1.3969775763038035
-1.3969775763038026
-1.3969775763038015
-1.396977576303801
-1.3943068147708542
-1.394306814770853
-1.3943068147708522
-1.3943068147708508
-1.3783359217715396
-1.3783359217715394
-1.378335921771539
-1.378335921771539
-1.3783359217715352
-1.3753192152879952
-1.375319215287991
-1.375108135991099
-1.3661781942955946
-1.3661781942955926
-1.3661781942955917
-1.3661781942955906
-1.3513339101602553
-1.3513339101602537
-1.3513339101602535
-1.3513339101602522
-1.349305509499411
-1.3493055094994093
-1.3493055094994082
-1.3493055094994049
-1.344537066581295
-1.3445370665812932
-1.344537066581293
-1.3445370665812895
-1.3445370665812892
-1.3445370665812877
-1.3445370665812875
-1.344537066581287
-1.3439491366941498
-1.3386156285979935
-1.33861562

-0.4480747005234752
-0.4480747005234746
-0.444453870853048
-0.44445387085304794
-0.4434692579886313
-0.44346925798862763
-0.37651960478665075
-0.3685320010655817
-0.3555593911097173
-0.3555593911097167
-0.35555939110971285
-0.29261718084038657
-0.27225075275407884
-0.27225075275407806
-0.21726779796546747
-0.2172677979654662
-0.21726779796546267
-0.19930212651928517
-0.19930212651928494
-0.19930212651928467
-0.19930212651928458
-0.1993021265192835
-0.19930212651928278
-0.19915937435799488
-0.19915937435799358
-0.19915937435799227
-0.19915937435799033
-0.19127292310810662
-0.19127292310810312
-0.1912729231081009
-0.18372484936633432
-0.18372484936633354
-0.16692902091768724
-0.15905058253843835
-0.1590505825384354
-0.15905058253843465
-0.1590505825384333
-0.15905058253843318
-0.15905058253843166
-0.15779667438770956
-0.1577966743877091
-0.1503097329724353
-0.1503097329724311
-0.12773181576432016
-0.1277318157643184
-0.12612686154812833
-0.07318047232409479
-0.07318047232409337
-0.073180

## H<sub>4</sub>

### Using Jordan-WIgner transformations

In [27]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h4 = get_qubit_hamiltonian(mol='h4', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(lih))

H:
-3.934441956757901 [] +
0.04069606505875819 [X0 X1 X2] +
-0.0031369437172315585 [X0 X1 X2 X3 Y7 Y11] +
0.0016354047160371907 [X0 X1 X2 Y3 Y5] +
0.0037144848216505294 [X0 X1 X2 Z3] +
0.0017809712977740805 [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
0.0047911716564078485 [X0 X1 Z2 Y3 Y4 X5] +
-0.00129194416648752 [X0 X1 X3 X4 Y7 Y11] +
0.001095485931572669 [X0 X1 X3 Y4 Y5 Z6 Z7] +
0.00129194416648752 [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
0.0025623902894191175 [X0 X1 X3 Z4 Y5 Y6 Z7] +
8.04917124090585e-05 [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
0.0013724358788965785 [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
0.0009084689845792891 [X0 X1 X3 X6 Y7 Y11] +
-8.981652044146124e-05 [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
0.0009084689845792898 [X0 X1 X3 Y7 X8 Y11] +
-0.0009084689845792898 [X0 X1 X3 Y7 Y8 Z10 X11] +
-8.981652044146084e-05 [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
0.0008186524641378289 [X0 X1 X3 Y7 Z8 Y10 X11] +
0.017297111110348394 [X0 X1 X3 Y7 Z9 Y10 X11] +
-0.002519150880579575 [X0 X1 X3 Y7 X10 Y11] +
-0.0008144678357783922 [X0 X1 Y3 X

In [28]:
# Build matrix representation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h4)
h4_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in lih.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h4_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h4_matrix)
print("The ground state energy from S1: ")
obtain_PES('h4', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -1.915106549748574 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-7.7911871  -7.73656236 -7.72762207 -7.69187324 -7.68171932 -7.65133164
 -7.63100655 -7.61859548 -7.60499436 -7.59841184 -7.57999977 -7.57411272
 -7.54530705 -7.53408654 -7.52821857 -7.51513135 -7.50366914 -7.48709235
 -7.48547854 -7.47779742 -7.47515409 -7.4560657  -7.45037895 -7.43839526
 -7.42849688 -7.37941749 -7.35746495 -7.3373327  -7.32831191 -7.31949502
 -7.29910778 -7.25346918 -7.20882391 -7.14266369 -7.12380203 -7.1142644
 -7.10019722 -7.08055241 -7.0675236  -7.06206633 -7.05380786 -7.05172245
 -7.04263455 -7.03804821 -7.01655126 -6.99060646 -6.98822358 -6.98107465
 -6.96123808 -6.95300835 -6.94066916 -6.91911018 -6.89158587 -6.88515836
 -6.88481004 -6.55963508 -6.54932513 -6.51839963 -6.50731786 -6.46647551
 -6.45923238 -6.33904529 -6.30077806 -5.74091262 -5.73483019 -5.73209716
 -5.72837036 -5.72401078 -5.71893209 -5.71847938 -5.70096772 -5.69445433
 -

In [29]:
for i in range(len(eigvals)):
    print(eigvals[i])

-7.791187096764351
-7.736562358309818
-7.7276220728376375
-7.691873240852811
-7.6817193240822945
-7.651331640076854
-7.631006549131178
-7.618595484473012
-7.6049943597104015
-7.598411836083159
-7.579999773594975
-7.5741127228638945
-7.545307053812379
-7.534086535580428
-7.528218570085409
-7.515131347428493
-7.503669140203969
-7.487092349354524
-7.485478540352639
-7.477797423237444
-7.475154093777547
-7.456065698997489
-7.450378946221876
-7.438395261190137
-7.428496884479689
-7.379417486328757
-7.357464947988923
-7.337332699783262
-7.32831190550253
-7.319495019694961
-7.2991077772701
-7.253469184136257
-7.2088239104379515
-7.142663686543998
-7.123802028905477
-7.114264401444903
-7.100197220341351
-7.080552414329167
-7.067523598062314
-7.062066330055149
-7.053807858652075
-7.051722451864718
-7.042634548870535
-7.038048206443965
-7.016551261522782
-6.9906064608782135
-6.988223575979868
-6.981074649661167
-6.961238078400147
-6.9530083508995215
-6.940669164237211
-6.919110180966288
-6.89158

### Using Bravyi-Kitaev transformations

In [30]:
qubit_transf = 'bk' # transformations
h4 = get_qubit_hamiltonian(mol='h4', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(lih))

H:
-3.934441956757901 [] +
0.04069606505875819 [X0 X1 X2] +
-0.0031369437172315585 [X0 X1 X2 X3 Y7 Y11] +
0.0016354047160371907 [X0 X1 X2 Y3 Y5] +
0.0037144848216505294 [X0 X1 X2 Z3] +
0.0017809712977740805 [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
0.0047911716564078485 [X0 X1 Z2 Y3 Y4 X5] +
-0.00129194416648752 [X0 X1 X3 X4 Y7 Y11] +
0.001095485931572669 [X0 X1 X3 Y4 Y5 Z6 Z7] +
0.00129194416648752 [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
0.0025623902894191175 [X0 X1 X3 Z4 Y5 Y6 Z7] +
8.04917124090585e-05 [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
0.0013724358788965785 [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
0.0009084689845792891 [X0 X1 X3 X6 Y7 Y11] +
-8.981652044146124e-05 [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
0.0009084689845792898 [X0 X1 X3 Y7 X8 Y11] +
-0.0009084689845792898 [X0 X1 X3 Y7 Y8 Z10 X11] +
-8.981652044146084e-05 [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
0.0008186524641378289 [X0 X1 X3 Y7 Z8 Y10 X11] +
0.017297111110348394 [X0 X1 X3 Y7 Z9 Y10 X11] +
-0.002519150880579575 [X0 X1 X3 Y7 X10 Y11] +
-0.0008144678357783922 [X0 X1 Y3 X

In [31]:
# Build matrix representation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h4)
h4_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in lih.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h4_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h4_matrix)
print("The ground state energy from S1: ")
obtain_PES('h4', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -1.915106549748574 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-7.7911871  -7.73656236 -7.72762207 -7.69187324 -7.68171932 -7.65133164
 -7.63100655 -7.61859548 -7.60499436 -7.59841184 -7.57999977 -7.57411272
 -7.54530705 -7.53408654 -7.52821857 -7.51513135 -7.50366914 -7.48709235
 -7.48547854 -7.47779742 -7.47515409 -7.4560657  -7.45037895 -7.43839526
 -7.42849688 -7.37941749 -7.35746495 -7.3373327  -7.32831191 -7.31949502
 -7.29910778 -7.25346918 -7.20882391 -7.14266369 -7.12380203 -7.1142644
 -7.10019722 -7.08055241 -7.0675236  -7.06206633 -7.05380786 -7.05172245
 -7.04263455 -7.03804821 -7.01655126 -6.99060646 -6.98822358 -6.98107465
 -6.96123808 -6.95300835 -6.94066916 -6.91911018 -6.89158587 -6.88515836
 -6.88481004 -6.55963508 -6.54932513 -6.51839963 -6.50731786 -6.46647551
 -6.45923238 -6.33904529 -6.30077806 -5.74091262 -5.73483019 -5.73209716
 -5.72837036 -5.72401078 -5.71893209 -5.71847938 -5.70096772 -5.69445433
 -

In [32]:
for i in range(len(eigvals)):
    print(eigvals[i])

-7.791187096764351
-7.736562358309818
-7.7276220728376375
-7.691873240852811
-7.6817193240822945
-7.651331640076854
-7.631006549131178
-7.618595484473012
-7.6049943597104015
-7.598411836083159
-7.579999773594975
-7.5741127228638945
-7.545307053812379
-7.534086535580428
-7.528218570085409
-7.515131347428493
-7.503669140203969
-7.487092349354524
-7.485478540352639
-7.477797423237444
-7.475154093777547
-7.456065698997489
-7.450378946221876
-7.438395261190137
-7.428496884479689
-7.379417486328757
-7.357464947988923
-7.337332699783262
-7.32831190550253
-7.319495019694961
-7.2991077772701
-7.253469184136257
-7.2088239104379515
-7.142663686543998
-7.123802028905477
-7.114264401444903
-7.100197220341351
-7.080552414329167
-7.067523598062314
-7.062066330055149
-7.053807858652075
-7.051722451864718
-7.042634548870535
-7.038048206443965
-7.016551261522782
-6.9906064608782135
-6.988223575979868
-6.981074649661167
-6.961238078400147
-6.9530083508995215
-6.940669164237211
-6.919110180966288
-6.89158